## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPyDatabase.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Narsaq,GL,60.9167,-46.0500,37.29,99,100,7.94,light rain
1,1,Thompson,CA,55.7435,-97.8558,-3.84,77,75,2.30,broken clouds
2,2,Nikolskoye,RU,59.7035,30.7861,40.87,97,100,12.91,overcast clouds
3,3,Mwinilunga,ZM,-11.7358,24.4293,62.64,84,100,2.55,overcast clouds
4,4,Avinurme,EE,58.9822,26.8642,37.04,95,100,13.20,overcast clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would prefer for your trip? "))
max_temp = float(input("What is the maximum temperature you would prefer for your trip? "))

What is the minimum temperature you would prefer for your trip? 80
What is the maximum temperature you would prefer for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,San Fernando,PH,15.0286,120.6898,80.69,80,100,4.59,overcast clouds
10,10,Butaritari,KI,3.0707,172.7902,82.49,73,95,17.34,overcast clouds
21,21,Hilo,US,19.7297,-155.0900,83.28,93,90,0.00,moderate rain
39,39,Bubaque,GW,11.2833,-15.8333,83.59,70,94,10.58,overcast clouds
46,46,Neiafu,TO,-18.6500,-173.9833,84.88,70,75,5.75,broken clouds
48,48,Faanui,PF,-16.4833,-151.7500,82.38,69,10,17.72,clear sky
50,50,Itarema,BR,-2.9248,-39.9167,81.00,75,88,13.98,overcast clouds
58,58,Sola,VU,-13.8833,167.5500,83.07,79,75,5.75,broken clouds
62,62,Avarua,CK,-21.2078,-159.7750,82.45,78,36,10.36,scattered clouds
64,64,Bundaberg,AU,-24.8500,152.3500,80.22,92,90,4.00,overcast clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                88
City                   88
Country                88
Lat                    88
Lng                    88
Max Temp               88
Humidity               88
Cloudiness             88
Wind Speed             88
Current Description    88
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,San Fernando,PH,80.69,overcast clouds,15.0286,120.6898,
10,Butaritari,KI,82.49,overcast clouds,3.0707,172.7902,
21,Hilo,US,83.28,moderate rain,19.7297,-155.0900,
39,Bubaque,GW,83.59,overcast clouds,11.2833,-15.8333,
46,Neiafu,TO,84.88,broken clouds,-18.6500,-173.9833,
48,Faanui,PF,82.38,clear sky,-16.4833,-151.7500,
50,Itarema,BR,81.00,overcast clouds,-2.9248,-39.9167,
58,Sola,VU,83.07,broken clouds,-13.8833,167.5500,
62,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,
64,Bundaberg,AU,80.22,overcast clouds,-24.8500,152.3500,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,San Fernando,PH,80.69,overcast clouds,15.0286,120.6898,Maharlika Hostel
10,Butaritari,KI,82.49,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
21,Hilo,US,83.28,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
39,Bubaque,GW,83.59,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
46,Neiafu,TO,84.88,broken clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation
48,Faanui,PF,82.38,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
50,Itarema,BR,81.00,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
58,Sola,VU,83.07,broken clouds,-13.8833,167.5500,Leumerus Bungalows
62,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
64,Bundaberg,AU,80.22,overcast clouds,-24.8500,152.3500,Takalvan Motel


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [25]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))